In [1]:
import os
import json
import librosa
import numpy as np


### Test Model h5

In [14]:
from tensorflow.keras.models import load_model

# Load the model
loaded_model = load_model('/Users/danielporras/Musica/music_x/best_model.h5')

In [15]:
import os

folder_path = '/Users/danielporras/Musica/music_import/Music/Sky Blue'
audio_files = [f for f in os.listdir(folder_path) if f.endswith('.mp3') or f.endswith('.m4a')]

print(f"There are {len(audio_files)} audio files in the 'Black' folder.")

There are 170 audio files in the 'Black' folder.


In [16]:
# Get the output layer
output_layer = loaded_model.layers[-1]

# Print the number of classes
print('Number of classes:', output_layer.output_shape[1])

Number of classes: 7


In [6]:
from sklearn.preprocessing import LabelEncoder

# Assume 'labels' is a list of your labels
labels = ['Red', 'Blue', 'Green', 'Yellow', 'Black', 'Gray', 'Sky Blue', 'Orange']

encoder = LabelEncoder()
encoder.fit(labels)

print('Class labels:', encoder.classes_)

Class labels: ['Black' 'Blue' 'Gray' 'Green' 'Orange' 'Red' 'Sky Blue' 'Yellow']


In [17]:
# Get the output layer
output_layer = loaded_model.layers[-1]

# Print the number of classes
print('Number of classes:', output_layer.output_shape[1])

Number of classes: 7


In [18]:
def load_and_preprocess_songs(directory):
    song_features = []
    song_names = []  # List to store the names of the songs
    
    for filename in os.listdir(directory):
        if filename.endswith('.mp3') or filename.endswith('.m4a'):  # Assuming the songs are in .mp3 or .m4a format
            song_path = os.path.join(directory, filename)
            mfccs = extract_features(song_path)
            
            if mfccs is not None:  # Only append features if mfccs could be computed
                # Reshape the data to match the expected input shape
                mfccs = np.reshape(mfccs, (26, 1, 1))
                song_features.append(mfccs)
                song_names.append(filename)  # Store the name of the song
                
    # Convert list to numpy array
    song_features = np.array(song_features)

    # Ensure the data has the correct shape
    if len(song_features.shape) < 4:
        # Add a dimension for the batch size
        song_features = np.expand_dims(song_features, axis=0)

    return song_features, song_names

In [19]:
def extract_features(file_path):
    audio, sample_rate = librosa.load(file_path, res_type='kaiser_fast') 
    mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=26)
    mfccs_processed = np.mean(mfccs.T,axis=0)
     
    return mfccs_processed

In [20]:
# Define the directory with the test songs
test_dir = '/Users/danielporras/Musica/music_import/test_songs'

# Load and preprocess the songs
test_songs, test_song_names = load_and_preprocess_songs(test_dir)

# Use the loaded model to make predictions
predictions = loaded_model.predict(test_songs)

# Get the class with the highest probability
predicted_classes = np.argmax(predictions, axis=1)

# Define the mapping from indices to class labels
class_labels = ['Red', 'Blue', 'Green', 'Yellow', 'Black', 'Gray', 'Sky Blue']

# Print the predicted classes along with the song names
for song_name, predicted_class in zip(test_song_names, predicted_classes):
    print(f'Song: {song_name}, Predicted class: {class_labels[predicted_class]}')

/var/folders/_t/p3lht79d2_jd6gwjl515j1g40000gn/T/ipykernel_25958/3396950221.py:2: UserWarning: PySoundFile failed. Trying audioread instead.
  audio, sample_rate = librosa.load(file_path, res_type='kaiser_fast')


1/1 [==============================] - 0s 119ms/step
Song: watermarked_AEROPLANES_Groovy_Town_instrumental_2_31 copy.mp3, Predicted class: Sky Blue
Song: watermarked_Music_City_Reggae_Squad_Steel_Pan_Island_instrumental_2_36.mp3, Predicted class: Sky Blue
Song: Buscando-el-Son-no-voice_AdobeStock_709714291_preview copy.m4a, Predicted class: Sky Blue
Song: watermarked_Music_City_Reggae_Squad_Steel_Pan_Island_instrumental_2_36 copy.mp3, Predicted class: Sky Blue
Song: watermarked_LNDO_Just_Look_Up_background_vocals_2_45 copy.mp3, Predicted class: Sky Blue
Song: watermarked_Jed_Stark_Holidaze_instrumental_2_03.mp3, Predicted class: Sky Blue
Song: watermarked_Material_Gurl_The_Happiest_Memories_instrumental_1_56.mp3, Predicted class: Sky Blue
Song: Spanish-Summer-Dance_AdobeStock_452607663_preview copy.m4a, Predicted class: Sky Blue
Song: watermarked_Reveille_Go_For_It_instrumental_2_48 copy.mp3, Predicted class: Sky Blue
Song: watermarked_Tiger_Gang_Hotpants_instrumental_1_20.mp3, Predict

In [19]:
num_classes = loaded_model.output_shape[-1]
print(f'The model can predict {num_classes} different classes.')

The model can predict 10 different classes.
